## Code#2 Information
<small> <quote>
**Author:**  Shafagh Keyvanian [shkey@seas.upenn.edu]  <br>
**Date**: *Spring 2024*
</quote> </small>

<Large> <quote>
**All Subjects**<br>
**Bones: Hip, Ab, Chest, UArm, FArm, Hand** </quote> </Large>

Input:  
.npz file containing Upper-body Euler angles
- Folder: 
<small> <blockquote> ../data/prossesed_exports/ </blockquote> </small>
- Columns:  
<small> <blockquote> 'time', 'eul_right', 'eul_left', 'eul_head' </blockquote> </small>
- Chains:  
<small> <blockquote>
        chain_R = ['Skeleton', 'Ab', 'Chest', 'RUArm','RFArm','RHand']<br>
        chain_L = ['Skeleton', 'Ab', 'Chest', 'LUArm','LFArm','LHand']<br>
        chain_H = ['Skeleton', 'Ab', 'Chest', 'Neck', 'Head'] </blockquote> </small>
    
Output: 
- arms_dof 
<small> <blockquote> ['sh_abd', 'sh_rot', 'sh_ext', 'el_fle', 'el_sup', 'wr_dev', 'wr_sup', 'wr_fle'] </blockquote> </small>
- Data diagnostics
- Plots

In [ ]:
#%% import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from tf.transformations import quaternion_matrix, euler_from_matrix, euler_matrix

In [ ]:
#%% Input data

# Initialize variables
n_dof = 8    
dof_names = ['Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension', 'Elbow Flexion', \
             'Elbow Supination', 'Wrist Deviation', 'Wrist Supination', 'Wrist Flexion']
dof_codes = ['sh_abd', 'sh_rot', 'sh_ext', 'el_fle', 'el_sup', 'wr_dev', 'wr_sup', 'wr_fle']

n_subjects = 5
subject_labels = ['Subject 0', 'Subject 1', 'Subject 2', 'Subject 3', 'Subject 4', 'All Subjects']

# Open npz file for each subject
npz_file_0 = '../data/processed_exports/subject0_7dof_eul_angles.npz'
npz_file_1 = '../data/processed_exports/subject1_7dof_eul_angles.npz'
npz_file_2 = '../data/processed_exports/subject2_7dof_eul_angles.npz'
npz_file_3 = '../data/processed_exports/subject3_7dof_eul_angles.npz'
npz_file_4 = '../data/processed_exports/subject4_7dof_eul_angles.npz'

npz_data_0, npz_data_1, npz_data_2, npz_data_3, npz_data_4 = \
    np.load(npz_file_0), np.load(npz_file_1), np.load(npz_file_2), np.load(npz_file_3), np.load(npz_file_4)

for i in range(5):
    exec(f"time_{i}, eul_right_{i}, eul_left_{i} = \
         npz_data_{i}['time'], npz_data_{i}['eul_right'], npz_data_{i}['eul_left']")


#%% Data diagnostics
# time_4 = time_4[0:time_4.shape[0]-800]
# eul_right_4 = eul_right_4[0:eul_right_4.shape[0]-800]
# eul_left_4 = eul_left_4[0:eul_left_4.shape[0]-800]

# eul_right_0 = eul_right_0[np.r_[0:12700, 22300:34000, 37000:54500, 56700:58500, 60800:62400, 65400:75000, 79000:87000],:,:]
# eul_left_0 = eul_left_0[np.r_[0:12700, 22300:34000, 37000:54500, 56700:58500, 60800:62400, 65400:75000, 79000:87000],:,:]
# time_0 = time_0[np.r_[0:12700, 22300:34000, 37000:54500, 56700:58500, 60800:62400, 65400:75000, 79000:87000]]

# time_2 = time_2[np.r_[0:15100, 15800:]]
# eul_right_2 = eul_right_2[np.r_[0:15100, 15800:],:,:]
# eul_left_2 = eul_left_2[np.r_[0:15100, 15800:],:,:]



for s in range(n_subjects):
    # Remove jumps in the data
    for t in range(1, len(eval(f"time_{s}"))):
        right_diff = eval(f"np.abs(eul_right_{s}[t,3:]-eul_right_{s}[t-1,3:])")
        left_diff = eval(f"np.abs(eul_left_{s}[t,3:]-eul_left_{s}[t-1,3:])")
        if (right_diff > np.pi/20).any():
            exec(f"eul_right_{s}[t] = eul_right_{s}[t-1]")
        if (left_diff > np.pi/20).any():
            exec(f"eul_left_{s}[t] = eul_left_{s}[t-1]")

    # Arm angles (+ and - for right and left arm consistency)
    exec(f"R_sh_abd_{s} = -np.around(eul_right_{s}[:,3,0] * 180/np.pi, 2)")
    exec(f"R_sh_rot_{s} =  np.around(eul_right_{s}[:,3,1] * 180/np.pi, 2)")
    exec(f"R_sh_ext_{s} = -np.around(eul_right_{s}[:,3,2] * 180/np.pi, 2)")
    exec(f"R_el_fle_{s} =  np.around(eul_right_{s}[:,4,0] * 180/np.pi, 2)")
    exec(f"R_el_sup_{s} =  np.around(eul_right_{s}[:,4,1] * 180/np.pi, 2)")
    exec(f"R_wr_dev_{s} = -np.around(eul_right_{s}[:,5,0] * 180/np.pi, 2)")
    exec(f"R_wr_sup_{s} =  np.around(eul_right_{s}[:,5,1] * 180/np.pi, 2)")
    exec(f"R_wr_fle_{s} = -np.around(eul_right_{s}[:,5,2] * 180/np.pi, 2)")

    exec(f"L_sh_abd_{s} =  np.around(eul_left_{s}[:,3,0] * 180/np.pi, 2)")
    exec(f"L_sh_rot_{s} =  np.around(eul_left_{s}[:,3,1] * 180/np.pi, 2)")
    exec(f"L_sh_ext_{s} =  np.around(eul_left_{s}[:,3,2] * 180/np.pi, 2)")
    exec(f"L_el_fle_{s} = -np.around(eul_left_{s}[:,4,0] * 180/np.pi, 2)")
    exec(f"L_el_sup_{s} =  np.around(eul_left_{s}[:,4,1] * 180/np.pi, 2)")
    exec(f"L_wr_dev_{s} =  np.around(eul_left_{s}[:,5,0] * 180/np.pi, 2)")
    exec(f"L_wr_sup_{s} =  np.around(eul_left_{s}[:,5,1] * 180/np.pi, 2)")
    exec(f"L_wr_fle_{s} =  np.around(eul_left_{s}[:,5,2] * 180/np.pi, 2)")
    
    # Healthy arm angles:
    for d in range(n_dof):
        if s == 0 or s == 4:
            exec(f"{dof_codes[d]}_{s} = R_{dof_codes[d]}_{s}")
        else:
            exec(f"{dof_codes[d]}_{s} = L_{dof_codes[d]}_{s}")  

#%% min and max values for healthy arm angles
min_angles = np.zeros((n_subjects, n_dof))
max_angles = np.zeros((n_subjects, n_dof))

for s in range(n_subjects):
    for i, dof_code in enumerate(dof_codes):
        exec(f"min_angles[s,{i}] = min({dof_code}_{s})")
        exec(f"max_angles[s,{i}] = max({dof_code}_{s})")
dof_min, dof_max = np.min(min_angles, axis=0), np.max(max_angles, axis=0)

#%% Plot the data vs time for diagnostics
i_start = 0 
i_end = 5000
i_end_values = np.zeros(n_subjects, dtype=int)
fig0, ax0 = plt.subplots(5, 1, sharex=False, figsize=(12, 20))
for s in range(n_subjects):
    # i_end_values[s] = min(i_end, len(eval(f"time_{s}")))  # Store the value in the dictionary
    if i_start < len(eval(f"time_{s}")):
        i_end_values[s] = min(i_end, len(eval(f"time_{s}"))-1)  
        for d in range(n_dof):
            ax0[s].plot(eval(f"time_{s}")[i_start:i_end_values[s]], eval(f"{dof_codes[d]}_{s}")[i_start:i_end_values[s]], label=f'{dof_names[d]}')
            ax0[s].set(xlabel='Time (s)', ylabel=f'{subject_labels[s]} (deg)', xlim=(eval(f"time_{s}")[i_start], eval(f"time_{s}")[i_start]+(i_end-i_start)/100+4))
        ax0[s].legend()
        ax0[s].grid()
        ax0[s].xaxis.set_major_locator(plt.MultipleLocator(2))
    else:
        #remove data for subjects with no data
        ax0[s].axis('off')

In [ ]:
#%% Print min and max values for healthy arm angles

print("{:^9} {:^16} {:^16} {:^20} {:^17} {:^20} {:^17} {:^16} {:^16}".format\
      ("", "Shoulder", "Shoulder", "Shoulder", "Elbow", "Elbow", "Wrist", "Wrist", "Wrist"))
print("{:^9} {:^16} {:^16} {:^20} {:^17} {:^20} {:^17} {:^16} {:^16}".format\
      ("", "Abduction", "Rotation", "Extension", "Flexion", "Supination", "Deviation", "Supination", "Flexion"))
print("{:<10} {:<8} {:<8} {:<8} {:<8} {:<8} {:<11} {:<8} {:<8} {:<8} {:<11} {:<8} {:<8} {:<8} {:<8} {:<8} {:<12}".format\
      ("Subject", "min", "max", "min", "max", "min", "max", "min", "max", "min", "max", "min", "max", "min", "max", "min", "max"))
for s in range(len(subject_labels)-1):
    print("{:<9} {:<8} {:<8} {:<8} {:<8} {:<8} {:<11} {:<8} {:<8} {:<8} {:<11} {:<8} {:<8} {:<8} {:<8} {:<8} {:<12}".format("#"+str(s), \
      str(min_angles[s,0]), str(max_angles[s,0]), str(min_angles[s,1]), str(max_angles[s,1]), str(min_angles[s,2]), \
      str(max_angles[s,2]), str(min_angles[s,3]), str(max_angles[s,3]), str(min_angles[s,4]), str(max_angles[s,4]), \
      str(min_angles[s,5]), str(max_angles[s,5]), str(min_angles[s,6]), str(max_angles[s,6]), str(min_angles[s,7]), str(max_angles[s,7])))
print("{:<9} {:<8} {:<8} {:<8} {:<8} {:<8} {:<11} {:<8} {:<8} {:<8} {:<11} {:<8} {:<8} {:<8} {:<8} {:<8} {:<12}".format("All", \
    str(dof_min[0]), str(dof_max[0]), str(dof_min[1]), str(dof_max[1]), str(dof_min[2]), str(dof_max[2]), \
    str(dof_min[3]), str(dof_max[3]), str(dof_min[4]), str(dof_max[4]), str(dof_min[5]), str(dof_max[5]), \
    str(dof_min[6]), str(dof_max[6]), str(dof_min[7]), str(dof_max[7])))

# print("{:<20} {:<8} {:<8} {:<8} {:<8} {:<12} {:<8} {:<8} {:<8} {:<8} {:<8}".format\
#       ("DoF", "min_0", "min_1", "min_2", "min_3", "min_4", "max_0", "max_1", "max_2", "max_3", "max_4"))
# for i in range(len(dof_codes)):
#     print("{:<20} {:<8} {:<8} {:<8} {:<8} {:<12} {:<8} {:<8} {:<8} {:<8} {:<8}".format(dof_names[i], \
#       str(min(eval(f"{dof_codes[i]}_0"))), str(min(eval(f"{dof_codes[i]}_1"))), str(min(eval(f"{dof_codes[i]}_2"))), \
#       str(min(eval(f"{dof_codes[i]}_3"))), str(min(eval(f"{dof_codes[i]}_4"))), \
#       str(max(eval(f"{dof_codes[i]}_0"))), str(max(eval(f"{dof_codes[i]}_1"))), str(max(eval(f"{dof_codes[i]}_2"))), \
#       str(max(eval(f"{dof_codes[i]}_3"))), str(max(eval(f"{dof_codes[i]}_4")))))

In [ ]:
#%% Plot angles vs eachother for each subject in a 2 by 3 figure
subject_colors = ['b', 'r', 'g', 'c', 'm']
pad = 5

# # Shoulder Abduction vs Shoulder Rotation
# fig1, ax1 = plt.subplots(2, 3, figsize=(12, 6.5), num=f'all_7dof_{dof_names[0]} vs {dof_names[1]}')
# # fig1.subplots_adjust(wspace=0.2, hspace=0.2)
# fig1.suptitle(f'{dof_names[0]} (deg) vs {dof_names[1]} (deg) for each subject', fontsize='xx-large')
# # Create a big subplot covering the entire figure and set the labels on it
# big_ax = fig1.add_subplot(111, frameon=False)
# big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
# big_ax.set_xlabel(f'{dof_names[0]} (deg)', fontsize='x-large', labelpad=15)
# big_ax.set_ylabel(f'{dof_names[1]} (deg)', fontsize='x-large', labelpad=15)

# for s in range(n_subjects):
#     ax1[s//3, s%3].scatter(eval(f"{dof_codes[0]}_{s}"), eval(f"{dof_codes[1]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=0.5)
#     ax1[1,2].scatter(eval(f"{dof_codes[0]}_{s}"), eval(f"{dof_codes[1]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=1-s*0.2)
# for i in range(2):
#     for j in range(3):
#         ax1[i,j].set(title=subject_labels[i*3+j], aspect='equal', xlim=(axes_min[0]-pad, axes_max[0]+pad), ylim=(axes_min[1]-pad, axes_max[1]+pad))
#         # ax1[i,j].set(xlabel=f'{dof_names[0]} (deg)', ylabel=f'{dof_names[1]} (deg)', title=subject_labels[i*3+j])
#         # ax1[i,j].xaxis.set_major_locator(plt.MultipleLocator(30))
#         # ax1[i,j].yaxis.set_major_locator(plt.MultipleLocator(30))
# fig1.savefig(f'../data/figures/all_7dof_{dof_names[0]} vs {dof_names[1]}.png')

# # Shoulder Abduction vs Shoulder Extension
# fig2, ax2 = plt.subplots(2, 3, figsize=(12, 10), num=f'all_7dof_{dof_names[0]} vs {dof_names[2]}')
# fig2.suptitle(f'{dof_names[0]} (deg) vs {dof_names[2]} (deg) for each subject', fontsize='xx-large')
# big_ax = fig2.add_subplot(111, frameon=False)
# big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
# big_ax.set_xlabel(f'{dof_names[0]} (deg)', fontsize='x-large', labelpad=15)
# big_ax.set_ylabel(f'{dof_names[2]} (deg)', fontsize='x-large', labelpad=15)
# for s in range(n_subjects):
#     ax2[s//3, s%3].scatter(eval(f"{dof_codes[0]}_{s}"), eval(f"{dof_codes[2]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=0.5)
#     ax2[1,2].scatter(eval(f"{dof_codes[0]}_{s}"), eval(f"{dof_codes[2]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=1-s*0.2)
# for i in range(2):
#     for j in range(3):
#         ax2[i,j].set(title=subject_labels[i*3+j], aspect='equal', xlim=(dof_min[0]-pad, dof_max[0]+pad), ylim=(dof_min[2]-pad, dof_max[2]+pad))
# fig2.savefig(f'../data/figures/all_7dof_{dof_names[0]} vs {dof_names[2]}.png')

# # Shoulder Rotation vs Shoulder Extension
# fig3, ax3 = plt.subplots(2, 3, figsize=(12, 12), num=f'all_7dof_{dof_names[1]} vs {dof_names[2]}')
# fig3.suptitle(f'{dof_names[1]} (deg) vs {dof_names[2]} (deg) for each subject', fontsize='xx-large')
# big_ax = fig3.add_subplot(111, frameon=False)
# big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
# big_ax.set_xlabel(f'{dof_names[1]} (deg)', fontsize='x-large', labelpad=15)
# big_ax.set_ylabel(f'{dof_names[2]} (deg)', fontsize='x-large', labelpad=15)
# for s in range(n_subjects):
#     ax3[s//3, s%3].scatter(eval(f"{dof_codes[1]}_{s}"), eval(f"{dof_codes[2]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=0.5)
#     ax3[1,2].scatter(eval(f"{dof_codes[1]}_{s}"), eval(f"{dof_codes[2]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=1-s*0.2)
# for i in range(2):
#     for j in range(3):
#         ax3[i,j].set(title=subject_labels[i*3+j], aspect='equal', xlim=(dof_min[1]-pad, dof_max[1]+pad), ylim=(dof_min[2]-pad, dof_max[2]+pad))
# fig3.savefig(f'../data/figures/all_7dof_{dof_names[1]} vs {dof_names[2]}.png')

# # Elbow Flexion vs Shoulder Abduction
# fig4, ax4 = plt.subplots(2, 3, figsize=(12, 10), num=f'all_7dof_{dof_names[3]} vs {dof_names[0]}')
# fig4.suptitle(f'{dof_names[3]} (deg) vs {dof_names[0]} (deg) for each subject', fontsize='xx-large')
# big_ax = fig4.add_subplot(111, frameon=False)
# big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
# big_ax.set_xlabel(f'{dof_names[3]} (deg)', fontsize='x-large', labelpad=15)
# big_ax.set_ylabel(f'{dof_names[0]} (deg)', fontsize='x-large', labelpad=15)
# for s in range(n_subjects):
#     ax4[s//3, s%3].scatter(eval(f"{dof_codes[3]}_{s}"), eval(f"{dof_codes[0]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=0.5)
#     ax4[1,2].scatter(eval(f"{dof_codes[3]}_{s}"), eval(f"{dof_codes[0]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=1-s*0.2)
# for i in range(2):
#     for j in range(3):
#         ax4[i,j].set(title=subject_labels[i*3+j], aspect='equal', xlim=(dof_min[3]-pad, dof_max[3]+pad), ylim=(dof_min[0]-pad, dof_max[0]+pad))
# fig4.savefig(f'../data/figures/all_7dof_{dof_names[3]} vs {dof_names[0]}.png')

# # Elbow Flexion vs Shoulder Rotation
# fig5, ax5 = plt.subplots(2, 3, figsize=(12, 8), num=f'all_7dof_{dof_names[3]} vs {dof_names[1]}')
# fig5.suptitle(f'{dof_names[3]} (deg) vs {dof_names[1]} (deg) for each subject', fontsize='xx-large')
# big_ax = fig5.add_subplot(111, frameon=False)
# big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
# big_ax.set_xlabel(f'{dof_names[3]} (deg)', fontsize='x-large', labelpad=15)
# big_ax.set_ylabel(f'{dof_names[1]} (deg)', fontsize='x-large', labelpad=15)
# for s in range(n_subjects):
#     ax5[s//3, s%3].scatter(eval(f"{dof_codes[3]}_{s}"), eval(f"{dof_codes[1]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=0.5)
#     ax5[1,2].scatter(eval(f"{dof_codes[3]}_{s}"), eval(f"{dof_codes[1]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=1-s*0.2)
# for i in range(2):
#     for j in range(3):
#         ax5[i,j].set(title=subject_labels[i*3+j], aspect='equal', xlim=(dof_min[3]-pad, dof_max[3]+pad), ylim=(dof_min[1]-pad, dof_max[1]+pad))
# fig5.savefig(f'../data/figures/all_7dof_{dof_names[3]} vs {dof_names[1]}.png')

# Elbow Flexion vs Shoulder Extension 
fig6, ax6 = plt.subplots(2, 3, figsize=(12, 12), num=f'all_7dof_{dof_names[3]} vs {dof_names[2]}')
fig6.suptitle(f'{dof_names[3]} (deg) vs {dof_names[2]} (deg) for each subject', fontsize='xx-large')
big_ax = fig6.add_subplot(111, frameon=False)
big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
big_ax.set_xlabel(f'{dof_names[3]} (deg)', fontsize='x-large', labelpad=15)
big_ax.set_ylabel(f'{dof_names[2]} (deg)', fontsize='x-large', labelpad=15)
for s in range(n_subjects):
    ax6[s//3, s%3].scatter(eval(f"{dof_codes[3]}_{s}"), eval(f"{dof_codes[2]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=0.5)
    ax6[1,2].scatter(eval(f"{dof_codes[3]}_{s}"), eval(f"{dof_codes[2]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=1-s*0.2)
for i in range(2):
    for j in range(3):
        ax6[i,j].set(title=subject_labels[i*3+j], aspect='equal', xlim=(dof_min[3]-pad, dof_max[3]+pad), ylim=(dof_min[2]-pad, dof_max[2]+pad))
fig6.savefig(f'../data/figures/all_7dof_{dof_names[3]} vs {dof_names[2]}.png')

# # Elbow Flexion vs Elbow Supination
# fig7, ax7 = plt.subplots(2, 3, figsize=(12, 8), num=f'all_7dof_{dof_names[3]} vs {dof_names[4]}')
# fig7.suptitle(f'{dof_names[3]} (deg) vs {dof_names[4]} (deg) for each subject', fontsize='xx-large')
# big_ax = fig7.add_subplot(111, frameon=False)
# big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
# big_ax.set_xlabel(f'{dof_names[3]} (deg)', fontsize='x-large', labelpad=15)
# big_ax.set_ylabel(f'{dof_names[4]} (deg)', fontsize='x-large', labelpad=15)
# for s in range(n_subjects):
#     ax7[s//3, s%3].scatter(eval(f"{dof_codes[3]}_{s}"), eval(f"{dof_codes[4]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=0.5)
#     ax7[1,2].scatter(eval(f"{dof_codes[3]}_{s}"), eval(f"{dof_codes[4]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=1-s*0.2)
# for i in range(2):
#     for j in range(3):
#         ax7[i,j].set(title=subject_labels[i*3+j], aspect='equal', xlim=(dof_min[3]-pad, dof_max[3]+pad), ylim=(dof_min[4]-pad, dof_max[4]+pad))
# fig7.savefig(f'../data/figures/all_7dof_{dof_names[3]} vs {dof_names[4]}.png')

# Wrist Flexion vs Wrist Deviation
fig8, ax8 = plt.subplots(2, 3, figsize=(12, 5.5), num=f'all_7dof_{dof_names[7]} vs {dof_names[5]}')
fig8.suptitle(f'{dof_names[7]} (deg) vs {dof_names[5]} (deg) for each subject', fontsize='xx-large')
big_ax = fig8.add_subplot(111, frameon=False)
big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
big_ax.set_xlabel(f'{dof_names[7]} (deg)', fontsize='x-large', labelpad=15)
big_ax.set_ylabel(f'{dof_names[5]} (deg)', fontsize='x-large', labelpad=15)
for s in range(n_subjects):
    ax8[s//3, s%3].scatter(eval(f"{dof_codes[7]}_{s}"), eval(f"{dof_codes[5]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=0.5)
    ax8[1,2].scatter(eval(f"{dof_codes[7]}_{s}"), eval(f"{dof_codes[5]}_{s}"), label=f'subject {s}', color=subject_colors[s], alpha=1-s*0.2)
for i in range(2):
    for j in range(3):
        ax8[i,j].set(title=subject_labels[i*3+j], aspect='equal', xlim=(dof_min[7]-pad, dof_max[7]+pad), ylim=(dof_min[5]-pad, dof_max[5]+pad))
fig8.savefig(f'../data/figures/all_7dof_{dof_names[7]} vs {dof_names[5]}.png')


# ['sh_abd', 'sh_rot', 'sh_ext', 'el_fle', 'el_sup', 'wr_dev', 'wr_sup', 'wr_fle']

# plt.show()

In [ ]:
# %% Plot angles over time

# fig, ax = plt.subplots(4, 2, sharex=False, figsize=(14, 20))
# for i in range(4):
#     for j in range(2):
#         for s in range(n_subjects):
#             exec(f"ax[i,j].plot(time_{s}, {dof_codes[i+j*4]}_{s}, label='{dof_names[i+j*4]} subj{s}')")
#         ax[i,j].set(xlabel='Time (s)', ylabel=f'{dof_names[i+j*4]} (deg)')
#         ax[i,j].legend()
#         ax[i,j].grid()

i_start = 12000 
i_end = 13500
i_end_values = np.zeros(n_subjects, dtype=int)
fig0, ax0 = plt.subplots(5, 1, sharex=False, figsize=(12, 20))
for s in range(n_subjects):
    i_end_values[s] = min(i_end, len(eval(f"time_{s}")))  # Store the value in the dictionary
    if i_start < len(eval(f"time_{s}")):
        for d in range(n_dof):
            ax0[s].plot(eval(f"time_{s}")[i_start:i_end_values[s]], eval(f"{dof_codes[d]}_{s}")[i_start:i_end_values[s]], label=f'{dof_names[d]}')
            ax0[s].set(xlabel='Time (s)', ylabel=f'{subject_labels[s]} (deg)', xlim=(eval(f"time_{s}")[i_start], eval(f"time_{s}")[i_end]+4))
    ax0[s].legend()
    ax0[s].grid()